<a href="https://colab.research.google.com/github/Saksham-Gupta-off/WiDS_EmoSense/blob/master/Week_2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/WiDS2023/twitter_data.csv", error_bad_lines=False)
data.head()

<ipython-input-3-7e96553d572e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/drive/MyDrive/WiDS2023/twitter_data.csv", error_bad_lines=False)
Skipping line 8836: expected 4 fields, saw 5

Skipping line 535882: expected 4 fields, saw 7



,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [ ]:
def preprocess_text(X, tokenize = False):
    '''
    1. Removes punctuation from each sentence
    2. Removes URLs from words
    3. Performs Lower Casing
    4. Perform Tokenization
    5. Remove StopWords
    6. Perform Stemming
    7. Perform Lemmetization
    '''
    # translator = str.maketrans('', '', string.punctuation)
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # X = [text.translate(translator) for text in X]
    X = [re.sub(r'[.,!$(){}*%@]', '', text) for text in X]
    X = [re.sub(r'http\S+|www\S+|https\S+', '', text) for text in X]
    X = [text.lower() for text in X]
    X = [word_tokenize(text) for text in X]
    X = [[word for word in text if word not in stop_words] for text in X]
    X = [[stemmer.stem(word) for word in text] for text in X]
    X = [[lemmatizer.lemmatize(word) for word in text] for text in X]
    if not tokenize:
        X = [' '.join(tokens) for tokens in X]

    return X

In [ ]:
X = preprocess_text(list(data["SentimentText"]))
y = list(data["Sentiment"])

dt = {'Text': X, 'Label': y}
df = pd.DataFrame(dt)

df.head(), data.head()

(                                                Text  Label
 0                                     sad apl friend      0
 1                              miss new moon trailer      0
 2                                 omg alreadi 7:30 :      1
 3  omgaga im sooo im gunna cri 've dentist sinc 1...      0
 4                              think mi bf cheat t_t      0,
    ItemID  Sentiment SentimentSource  \
 0       1          0    Sentiment140   
 1       2          0    Sentiment140   
 2       3          1    Sentiment140   
 3       4          0    Sentiment140   
 4       5          0    Sentiment140   
 
                                        SentimentText  
 0                       is so sad for my APL frie...  
 1                     I missed the New Moon trail...  
 2                            omg its already 7:30 :O  
 3            .. Omgaga. Im sooo  im gunna CRy. I'...  
 4           i think mi bf is cheating on me!!!   ...  )

In [ ]:
df.to_pickle('/content/drive/MyDrive/WiDS2023/dataframe.pickle')

In [ ]:
loaded_df = pd.read_pickle('/content/drive/MyDrive/WiDS2023/dataframe.pickle')
X = list(loaded_df["Text"])
y = list(loaded_df["Label"])
loaded_df.head()

,Text,Label
0,sad apl friend,0
1,miss new moon trailer,0
2,omg alreadi 7:30 :,1
3,omgaga im sooo im gunna cri 've dentist sinc 1...,0
4,think mi bf cheat t_t,0


### Question 1.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize text data and convert it into sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to ensure uniform length input for the model
max_sequence_length = 100  # Adjust as needed based on your data
padded_sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_length))
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.3, random_state=42)

# Build the LSTM model
embedding_dim = 100  # Dimension of word embeddings
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128))  # LSTM layer with 128 units
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/5
15540/15540 [==============================] - 388s 25ms/step - loss: 0.4571 - accuracy: 0.7828 - val_loss: 0.4385 - val_accuracy: 0.7951
Epoch 2/5
15540/15540 [==============================] - 353s 23ms/step - loss: 0.3122 - accuracy: 0.8670 - val_loss: 0.4958 - val_accuracy: 0.7714
Epoch 3/5
15540/15540 [==============================] - 352s 23ms/step - loss: 0.2301 - accuracy: 0.9033 - val_loss: 0.5725 - val_accuracy: 0.7656
Epoch 4/5
15540/15540 [==============================] - 356s 23ms/step - loss: 0.1851 - accuracy: 0.9219 - val_loss: 0.6579 - val_accuracy: 0.7617
Epoch 5/5
14800/14800 [==============================] - 63s 4ms/step - loss: 0.7230 - accuracy: 0.7529
Test Accuracy: 75.29%


In [ ]:
model.save('/content/drive/MyDrive/WiDS2023/week2_lstm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the HDF5 file
loaded_model = load_model('/content/drive/MyDrive/WiDS2023/week2_lstm.h5')

In [ ]:
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

14800/14800 [==============================] - 70s 5ms/step - loss: 0.7230 - accuracy: 0.7529
Test Accuracy: 75.29%


### Question 2.

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Word2vec
import gensim

# Utility
import re
import os
from collections import Counter
import logging
import time
import pickle
import itertools
import gc
import json
from keras.preprocessing.text import tokenizer_from_json
from keras.models import model_from_json
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_pickle('/content/drive/MyDrive/WiDS2023/dataframe.pickle')
data.head()

,Text,Label
0,sad apl friend,0
1,miss new moon trailer,0
2,omg alreadi 7:30 :,1
3,omgaga im sooo im gunna cri 've dentist sinc 1...,0
4,think mi bf cheat t_t,0


In [5]:
documents = [_text.split() for _text in data.Text]

In [6]:
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE,
                                            window=W2V_WINDOW,
                                            min_count=W2V_MIN_COUNT,
                                            workers=8)

w2v_model.build_vocab(documents)

In [7]:
words = w2v_model.wv.key_to_index.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 36989


In [8]:
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

(347898836, 432507488)

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.Text)
vocab_size = len(tokenizer.word_index)+1
print('Vocab Size is ',vocab_size)

Vocab Size is  657892


In [10]:
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

In [11]:
x_data = pad_sequences(tokenizer.texts_to_sequences(data.Text) , maxlen = SEQUENCE_LENGTH)

In [12]:
y_data = data.Label
print(x_data.shape)
print(y_data.shape)
y_data = y_data.values.reshape(-1,1)

(1578612, 300)
(1578612,)


In [13]:
embedding_matrix = np.zeros((vocab_size , W2V_SIZE))
for word , i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(657892, 300)


In [14]:
embedding_layer = Embedding( vocab_size , W2V_SIZE , weights = [embedding_matrix] , input_length = SEQUENCE_LENGTH, trainable = False)
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128 , dropout = 0.2 , recurrent_dropout = 0.2 ,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, dropout = 0.2 , recurrent_dropout = 0.2 ))
model.add(Dropout(0.1))
model.add(Dense(1,activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          197367600 
                                                                 
 lstm (LSTM)                 (None, 300, 128)          219648    
                                                                 
 dropout (Dropout)           (None, 300, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 197636721 (753.92 MB)
Trainable params: 26

In [15]:
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

In [ ]:
%%time
history = model.fit(X_train , y_train , batch_size = BATCH_SIZE , epochs = EPOCHS , validation_split = 0.1  , verbose = 1 , callbacks = callbacks)

Epoch 1/8
972/972 [==============================] - ETA: 0s - loss: 0.4675 - accuracy: 0.7763

972/972 [==============================] - 1940s 2s/step - loss: 0.4675 - accuracy: 0.7763 - val_loss: 0.4430 - val_accuracy: 0.7910 - lr: 0.0010
Epoch 2/8
751/972 [======================>.......] - ETA: 7:12 - loss: 0.4424 - accuracy: 0.7915

In [ ]:
def predict(text):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]

    return {"score": float(score),
       "elapsed_time": time.time()-start_at}

In [ ]:
model.save('/content/drive/MyDrive/WiDS2023/week2_w2v_lstm.h5')
tokenizer_json = tokenizer.to_json()
with open('/content/drive/MyDrive/WiDS2023/week2_w2v_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [ ]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model

# Load the model from the HDF5 file
loaded_model = load_model('/content/drive/MyDrive/WiDS2023/week2_w2v_lstm.h5')

# Load tokenizer from JSON
with open('/content/drive/MyDrive/WiDS2023/week2_w2v_tokenizer.json', 'r', encoding='utf-8') as f:
    tokenizer_json = json.load(f)
    tokenizer = Tokenizer.from_json(tokenizer_json)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
import pickle

# Save 'history' to a pickle file
with open('/content/drive/MyDrive/WiDS2023/week2_q2_history.pkl', 'wb') as file:
    pickle.dump(history, file)

### Question 3.

In [ ]:
import pickle

# Load 'history' from the pickle file
with open('/content/drive/MyDrive/WiDS2023/week2_q2_history.pkl', 'rb') as file:
    history = pickle.load(file)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create and train a logistic regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Evaluate logistic regression model
logistic_accuracy = logistic_model.score(X_test, y_test)
print(f"Logistic Regression Test Accuracy: {logistic_accuracy * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Accessing loss and accuracy data from the history objects
lstm_loss = history.history['loss']
lstm_accuracy = history.history['accuracy']
lstm_val_loss = history.history['val_loss']
lstm_val_accuracy = history.history['val_accuracy']

# Accessing loss and accuracy data from the logistic regression
# As logistic regression doesn't have epochs, just providing a single point on the plot
logistic_loss = [logistic_model.score(X_test, y_test)]
logistic_accuracy = [logistic_accuracy]

# Plotting Loss vs Epoch
plt.figure(figsize=(10, 5))
plt.plot(lstm_loss, label='LSTM Training Loss')
plt.plot(lstm_val_loss, label='LSTM Validation Loss')
plt.plot(logistic_loss, marker='o', linestyle='', label='Logistic Regression Loss', color='red')
plt.title('Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plotting Accuracy vs Epoch
plt.figure(figsize=(10, 5))
plt.plot(lstm_accuracy, label='LSTM Training Accuracy')
plt.plot(lstm_val_accuracy, label='LSTM Validation Accuracy')
plt.plot(logistic_accuracy, marker='o', linestyle='', label='Logistic Regression Accuracy', color='red')
plt.title('Accuracy vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Question 4.

In [ ]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model

from google.colab import drive
drive.mount('/content/drive')
# Load the model from the HDF5 file
loaded_model = load_model('/content/drive/MyDrive/WiDS2023/week2_w2v_lstm.h5')

# Load tokenizer from JSON
with open('/content/drive/MyDrive/WiDS2023/week2_w2v_tokenizer.json', 'r', encoding='utf-8') as f:
    tokenizer_json = json.load(f)
    tokenizer = Tokenizer.from_json(tokenizer_json)

In [ ]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences

# Load the test dataset
test_dataset_path = '/content/drive/MyDrive/WiDS2023/test.csv'
test_df = pd.read_csv(test_dataset_path)

# Assuming 'post_text' is the column containing text data in the test dataset
texts = test_df['post_text'].tolist()

# Tokenize text and generate predictions
max_sequence_length = 100  # Assuming the same sequence length used during model training
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Predict sentiment scores for the test data
predictions = loaded_model.predict(padded_sequences)

# Assigning labels based on sentiment score threshold (adjust as needed)
threshold = 0.5  # Adjust this threshold according to your needs
labels = ['Positive' if score > threshold else 'Negative' for score in predictions]

# Add the predicted labels to the DataFrame
test_df['label'] = labels

# Save the DataFrame with predicted labels as a CSV file
output_file_path = '/content/drive/MyDrive/WiDS2023/week2_q4_output.csv'
test_df.to_csv(output_file_path, index=False)

In [1]:
X_test

NameError: ignored